# Processing Codenames Clues

In [2]:
from itertools import chain, combinations
import gensim

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin', binary=True, limit=500000
)

In [4]:
def powerset(iterable):
    s = list(iterable)
    pwrset = list(chain.from_iterable(combinations(s, r) for r in range(len(s))))
    pwrset.remove(())
    return pwrset

In [5]:
blue = ['piano', 'ambulance', 'bugle', 'missile', 'bond', 'heart', 'crane', 'smuggler', 'bank']
red = ['shadow', 'vacuum', 'root', 'ham', 'head', 'march', 'ray', 'air']
bomb = ['Beijing']

In [6]:
model.most_similar(
    positive=blue,
    negative=(red+bomb),
    restrict_vocab=50000)

[('paramedic', 0.355133056640625),
 ('defibrillator', 0.33805346488952637),
 ('Ambulance_Service', 0.33499324321746826),
 ('violin', 0.3247973322868347),
 ('Ambulance', 0.3132508397102356),
 ('cello', 0.3054354190826416),
 ('tow_truck', 0.29749977588653564),
 ('EMT', 0.2931399345397949),
 ('volunteer_firefighter', 0.291475772857666),
 ('cruiser', 0.2910142242908478)]

In [7]:
combinations_red = powerset(red)[9:]

clue_options_red = [(subset_red, model.most_similar(
    positive=subset_red,
    restrict_vocab=50000)[0]) for subset_red in combinations_red]

list(sorted(clue_options_red, key=lambda r: r[1][1], reverse=True))

[(('shadow', 'ray'), ('shadows', 0.5974277853965759)),
 (('head', 'march'), ('marching', 0.5897948145866394)),
 (('march', 'air'), ('marches', 0.5881889462471008)),
 (('root', 'march'), ('marches', 0.5844696164131165)),
 (('shadow', 'march'), ('marching', 0.5632286071777344)),
 (('shadow', 'root'), ('shadows', 0.5623012185096741)),
 (('shadow', 'vacuum', 'ray'), ('shadows', 0.5572865009307861)),
 (('shadow', 'root', 'ray'), ('shadows', 0.5564109086990356)),
 (('march', 'ray'), ('marches', 0.5475905537605286)),
 (('shadow', 'vacuum', 'root', 'ray'), ('shadows', 0.534398078918457)),
 (('shadow', 'ray', 'air'), ('shadows', 0.5296165943145752)),
 (('ham', 'march'), ('marching', 0.5274919271469116)),
 (('ham', 'head'), ('turkey', 0.5244491696357727)),
 (('vacuum', 'march'), ('marching', 0.5238999128341675)),
 (('shadow', 'vacuum', 'root'), ('shadows', 0.520523190498352)),
 (('root', 'march', 'air'), ('marches', 0.5203446745872498)),
 (('shadow', 'air'), ('shadows', 0.5177650451660156)),
 ((

In [8]:
combinations_blue = powerset(blue)[9:]
clue_options_blue = [(subset_blue, model.most_similar(
    positive=subset_blue,
    restrict_vocab=50000)[1]) for subset_blue in combinations_blue]

list(sorted(clue_options_blue, key=lambda r: r[1][1], reverse=True))

[(('piano', 'bugle'), ('trombone', 0.7237838506698608)),
 (('piano', 'bugle', 'smuggler'), ('flute', 0.6708239912986755)),
 (('piano', 'bugle', 'crane'), ('cello', 0.6600197553634644)),
 (('piano', 'bugle', 'bond'), ('saxophone', 0.6559885144233704)),
 (('piano', 'bugle', 'heart'), ('saxophone', 0.6510331630706787)),
 (('ambulance', 'crane'), ('ambulances', 0.648966372013092)),
 (('piano', 'smuggler'), ('cello', 0.6421153545379639)),
 (('missile', 'bank'), ('missiles', 0.6379957795143127)),
 (('missile', 'bond'), ('ballistic_missile', 0.6330827474594116)),
 (('piano', 'crane'), ('cello', 0.6330048441886902)),
 (('piano', 'ambulance', 'bugle'), ('saxophone', 0.6324641704559326)),
 (('missile', 'crane'), ('ballistic_missile', 0.6304953098297119)),
 (('piano', 'bugle', 'bank'), ('saxophone', 0.6240891814231873)),
 (('piano', 'missile'), ('missiles', 0.6240473985671997)),
 (('piano', 'bugle', 'bond', 'smuggler'), ('flute', 0.6234050393104553)),
 (('bugle', 'missile'), ('ballistic_missile',

In [9]:
for i in range(20):
    print(combinations_red[i])
    print(model.most_similar(
        positive=combinations_red[i],
        restrict_vocab=50000))
    print()

('shadow', 'root')
[('shadows', 0.5623012185096741), ('rooted', 0.44796422123908997), ('rooting', 0.4167397618293762), ('roots', 0.4109833538532257), ('eradicate', 0.40142446756362915), ('blossom', 0.396065354347229), ('weed', 0.3929741382598877), ('shade', 0.3917708694934845), ('uproot', 0.3911558985710144), ('deep_rooted', 0.3862999677658081)]

('shadow', 'ham')
[('turkey', 0.45676305890083313), ('shadows', 0.455523818731308), ('sausage', 0.44157153367996216), ('chicken', 0.44087815284729004), ('pancake', 0.4376571476459503), ('bacon', 0.43761688470840454), ('mashed_potatoes', 0.42525404691696167), ('slices', 0.4032338559627533), ('pudding', 0.4015660285949707), ('supper', 0.3989819586277008)]

('shadow', 'head')
[('heads', 0.5060592293739319), ('shadows', 0.483318030834198), ('Head', 0.43438106775283813), ('assistant', 0.425337016582489), ('deputy', 0.4057390093803406), ('chair', 0.3996628522872925), ('arm', 0.39767056703567505), ('secretary', 0.3886992335319519), ('chief', 0.378526

In [76]:
def find_grouping_optimum(grouping, red, bomb):
    if len(grouping) == 1:
        grouping_options = model.similar_by_word(grouping[0], topn=10)
    else:
        try:
            grouping_options = model.most_similar(positive=grouping, negative=(red+bomb), restrict_vocab=50000)
        except:
            print(grouping)
    
    for option in grouping_options:
        if is_valid_option(grouping, option):
            return (grouping, option)
        
    return (grouping, ("null", 0))

def is_valid_option(grouping, option):
    chars = set('0123456789#$,')
    for clue in grouping:
        if clue.lower() in option[0].lower() or option[0].lower() in clue.lower() or any((c in option[0]) for c in chars):
            return False
    return True

def apply_weight(grouping_option):
    clue = grouping_option[1][0]
    score = grouping_option[1][1] * len(grouping_option[0])
    return (grouping_option[0], (clue, score))

def sort_clue_options(clue_options):
    return list(sorted(clue_options, key=lambda r: r[1][1], reverse=True))

def find_best_clues(clues, opposing_clues, bomb):
    groupings = []
    while len(clues) > 0:
        if len(clues) == 1:
            combination_clues = [(clues[0],)]
        else:
            combination_clues = powerset(clues)[len(clues):]
            
        clue_options = [find_grouping_optimum(grouping, red, bomb) for grouping in combination_clues]
        clue_options = list(map(apply_weight, clue_options))
        sorted_clue_options = sort_clue_options(clue_options)
    
        groupings.append(clue_options[0])
        for clue in clue_options[0][0]:
            clues.remove(clue)
    return groupings

In [77]:
blue = ['piano', 'ambulance', 'bugle', 'missile', 'bond', 'heart', 'crane', 'smuggler', 'bank']
red = ['shadow', 'vacuum', 'root', 'ham', 'head', 'march', 'ray', 'air']
bomb = ['Beijing']

find_best_clues(blue, red, bomb)

[(('piano', 'ambulance'), ('EMT', 0.6792808175086975)),
 (('bugle', 'missile'), ('Cephalon', 0.505206286907196)),
 (('bond', 'heart'), ('Relationships', 0.7115859985351562)),
 (('crane', 'smuggler'), ('Cephalon', 0.5428393483161926)),
 (('bank',), ('lender', 0.6342284679412842))]